### Классификация текстов: спам-фильтр для SMS
В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB

**1)** Загрузите датасет. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [2]:
cd C:\Users\vlad\Machine_Learning\Yandex Specialization\5.Applied Tasks Of Data Analysis\data

C:\Users\vlad\Machine_Learning\Yandex Specialization\5.Applied Tasks Of Data Analysis\data


In [3]:
data = pd.read_csv('SMSSpamCollection.txt', sep='\t', header=None, names=['class', 'text'])
data['is_spam'] = data['class'].map({'ham':0,'spam':1})
data.head()

,class,text,is_spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [4]:
print('Texts Size: ', data['text'].shape[0])

Texts Size:  5572


**2)** Используя ```sklearn.feature_extraction.text.CountVectorizer``` со стандартными настройками, получите из списка текстов матрицу признаков ```x```

In [5]:
vectorizer = CountVectorizer(ngram_range=(1,1))
feature_matrix = vectorizer.fit_transform(data['text'])
print('Number of Features: ', feature_matrix.shape[1])

Number of Features:  8713


**3)** Оцените качество классификации текстов с помощью ```LogisticRegression()``` с параметрами по умолчанию, используя ```sklearn.cross_validation.cross_val_score``` и посчитав среднее арифметическое качества на отдельных фолдах.

Установите ```random_state=2```. Параметр ```cv``` задайте равным 10. В качестве метрики качества используйте ```f1-меру```. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [6]:
np.random.seed(2)

log_reg_model = LogisticRegression(random_state=2)

cv_score = cross_val_score(log_reg_model, feature_matrix, data['is_spam'], scoring='f1', cv=10, n_jobs=-1).mean()
print('Avg F1 Score Using 10 Folds: %.4f' %cv_score)

Avg F1 Score Using 10 Folds: 0.9312


In [7]:
cd C:\Users\vlad\Machine_Learning\Yandex Specialization\5.Applied Tasks Of Data Analysis\submissions

C:\Users\vlad\Machine_Learning\Yandex Specialization\5.Applied Tasks Of Data Analysis\submissions


In [8]:
# Определим функции для сохранения скаляров и списков значений
def save_value(f_name, value):
    with open(f_name, 'w') as f:
        f.write(str(value))
        
def save_value_list(f_name, values):
    with open(f_name, 'w') as f:
        f.write(' '.join([str(value) for value in values]))
    
    
save_value('spam_classification_1.txt', round(cv_score,1))

**4**) А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:
- "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"
- "FreeMsg: Txt: claim your reward of 3 hours talk time"
- "Have you visited the last lecture on physics?"
- "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

- "Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [9]:
# Обучаем
log_reg_model.fit(feature_matrix, data['is_spam'])

# Протестируем классификатор на примерах
texts = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"
]


results = []
for text in texts:
    print(text)
    pred = log_reg_model.predict(vectorizer.transform([text]))[0]
    results.append(pred)
    if pred == 1:
        print('SPAM')
    else:
        print('HAM')
    print()
    
# Сохраняем результаты
save_value_list('spam_classification_2.txt', results)

FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB
SPAM

FreeMsg: Txt: claim your reward of 3 hours talk time
SPAM

Have you visited the last lecture on physics?
HAM

Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$
HAM

Only 99$
HAM



**5)** Задайте в ```CountVectorizer``` параметр ```ngram_range=(2,2)```, затем ```ngram_range=(3,3)```, затем ```ngram_range=(1,3)```. Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке.

В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. **Обратите внимание**, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [10]:
np.random.seed(2)

n_grams_params = [(2,2), (3,3), (1,3)] 
log_reg_model = LogisticRegression(random_state=2)
results = []

for n_grams_param in n_grams_params:
    vectorizer = CountVectorizer(ngram_range=n_grams_param)
    feature_matrix = vectorizer.fit_transform(data['text'])
    print('Number of Features: ', feature_matrix.shape[1])
    cv_score = cross_val_score(log_reg_model, feature_matrix, data['is_spam'], scoring='f1', cv=10, n_jobs=-1).mean()
    print('Avg F1 Score Using 10 Folds: %.4f' %cv_score)
    results.append(round(cv_score, 2))
    print()

Number of Features:  41793
Avg F1 Score Using 10 Folds: 0.8169

Number of Features:  54461
Avg F1 Score Using 10 Folds: 0.7250

Number of Features:  104967
Avg F1 Score Using 10 Folds: 0.9217



Действительно, качество для триграмм заметно хуже (хоть признаков и много, но полезной статистики мало). Идельным оказался вариант комбинирования униграмм и биграмм.

In [11]:
save_value_list('spam_classification_3.txt', results)

**6)** Аналогично предыдущему эксперименту используйте вместо логистической регрессии ```MultinomialNB()```.

Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [12]:
np.random.seed(2)

mul_nb_classifier = MultinomialNB()
n_grams_params = [(2,2), (3,3), (1,3)]
results = []

for n_grams_param in n_grams_params:
    vectorizer = CountVectorizer(ngram_range=n_grams_param)
    feature_matrix = vectorizer.fit_transform(data['text'])
    print('Number of Features: ', feature_matrix.shape[1])
    cv_score = cross_val_score(mul_nb_classifier, feature_matrix, data['is_spam'], scoring='f1', cv=10, n_jobs=-1).mean()
    print('Avg F1 Score Using 10 Folds: %.4f' %cv_score)
    results.append(round(cv_score, 2))
    print()

Number of Features:  41793
Avg F1 Score Using 10 Folds: 0.6458

Number of Features:  54461
Avg F1 Score Using 10 Folds: 0.3786

Number of Features:  104967
Avg F1 Score Using 10 Folds: 0.8885



In [13]:
save_value_list('spam_classification_4.txt', results)

Заметно, что качество наивного байеса заметно ниже для биграмм и триграмм. Классификатор явно хуже справляется с задачей, чем логистическая регрессия.

**7)** Попробуйте использовать в логистической регрессии в качестве признаков ```Tfidf``` из ```TfidfVectorizer``` на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с ```CountVectorizer``` на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). 

Обратите внимание, что результат перехода к ```tfidf``` не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [14]:
np.random.seed(2)

log_reg_model = LogisticRegression(random_state=2)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))

feature_matrix = tfidf_vectorizer.fit_transform(data['text'])

cv_score = cross_val_score(log_reg_model, feature_matrix, data['is_spam'], scoring='f1', cv=10, n_jobs=-1).mean()
print('Avg F1 Score Using 10 Folds: %.4f' %cv_score)

Avg F1 Score Using 10 Folds: 0.8520


Основное различие между TfidfVectorizer и TfidfTransformer:
- TfidfVectorizer комбинирует в себе CountVectorizer и TfidfTransformer (т.е. сначала получает частоты слов, затем значения tf-idf)
- TfidfTransformer применяется к матрице частот слов.

In [15]:
save_value('spam_classification_5.txt', -1)